# Proyecto Parcial 1 - MLY0100
**Tema:** Predicción y Clasificación de Gasto de Clientes en un Supermercado  
**Integrantes:** Antonio Sepúlveda.
**Fecha:** 16/09/2025


# Entendimiento del Negocio

El objetivo de este análisis es segmentar a los clientes de una empresa minorista utilizando el dataset "Customer Personality Analysis" de Kaggle. Esto permitirá identificar patrones de comportamiento y características clave para mejorar estrategias de marketing y personalización de servicios.

**Caso real:**  
Las empresas minoristas buscan comprender mejor a sus clientes para ofrecer productos y promociones personalizadas, optimizando así la satisfacción y la rentabilidad.

- El dataset contiene información demográfica, económica y de comportamiento de compra de los clientes.(codigo #1)
- Número de filas y columnas(codigo #2)

In [ ]:
#Codigo #1
# Análisis exploratorio de datos y preprocesamiento

import pandas as pd

# Cargar datos
df = pd.read_csv('../data/marketing_campaign.csv', sep='\t')

# Mostrar primeras filas
df.head()

In [ ]:
#codigo #2
# Número de filas y columnas
print(f"Filas: {df.shape[0]}, Columnas: {df.shape[1]}")
df.info()

Aquí observamos la media, mediana, desviación estándar, valores mínimos y máximos de cada variable numérica.

In [ ]:
# Estadísticas descriptivas de variables numéricas
df.describe()

Visualización de Distribuciones

Los histogramas muestran la frecuencia de los valores en cada variable. Así identificamos si hay sesgos o valores extremos.

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Histograma de variables numéricas principales
df.hist(figsize=(15,10), bins=20)
plt.tight_layout()
plt.show()

Correlación entre Variables para ver qué variables numéricas están relacionadas entre sí.

El mapa de calor muestra la fuerza de la relación entre variables numéricas. Valores cercanos a 1 o -1 indican alta correlación.

In [ ]:
# Mapa de calor de correlación
plt.figure(figsize=(25,20))
sns.heatmap(df.corr(numeric_only=True), annot=True, cmap='coolwarm')
plt.title('Matriz de correlación')
plt.show()

Distribución de Variables Categóricas para ver la cantidad de observaciones en cada categoría.

Así visualizamos el balance de las categorías principales.

In [ ]:
# Conteo de categorías en Education y Marital_Status
print(df['Education'].value_counts())
print(df['Marital_Status'].value_counts())

# Gráficos de barras
fig, axes = plt.subplots(1, 2, figsize=(12,5))
sns.countplot(data=df, x='Education', ax=axes[0])
axes[0].set_title('Distribución de Education')
sns.countplot(data=df, x='Marital_Status', ax=axes[1])
axes[1].set_title('Distribución de Marital_Status')
plt.tight_layout()
plt.show()

Revisión y tratamiento de valores faltantes.
Los valores faltantes pueden afectar el análisis y los modelos. Debes identificarlos y decidir cómo tratarlos.

Aquí identificamos columnas con valores faltantes. Si hay pocos, podemos eliminarlos; si son muchos, podemos imputar (rellenar) con la media, mediana o moda según el tipo de variable.

In [ ]:
# Ver cuántos valores faltantes hay por columna
df.isnull().sum()

Limpieza de variables categóricas (errores y rarezas).
Algunas categorías como "YOLO", "Absurd" o "Alone" no son válidas o son muy raras.

Agrupamos categorías raras en "Other" para evitar que afecten el análisis.

In [ ]:
# Revisar valores únicos
print(df['Education'].unique())
print(df['Marital_Status'].unique())

# Reemplazar o agrupar valores raros
df['Education'] = df['Education'].replace({'2n Cycle': '2nd Cycle'})
df['Marital_Status'] = df['Marital_Status'].replace(
    {'Absurd': 'Other', 'YOLO': 'Other', 'Alone': 'Other'}
)

Detección y tratamiento de outliers. 
Los valores extremos pueden distorsionar el análisis y los modelos.

Detectamos outliers en variables numéricas importantes y los eliminamos para evitar distorsiones.

 (WRNNG: no ejecutar para conservar toda la info)

In [ ]:
import numpy as np

# Ejemplo: detectar outliers en Income usando IQR
Q1 = df['Income'].quantile(0.25)
Q3 = df['Income'].quantile(0.75)
IQR = Q3 - Q1
outliers = df[(df['Income'] < (Q1 - 1.5 * IQR)) | (df['Income'] > (Q3 + 1.5 * IQR))]
print(f"Outliers en Income: {outliers.shape[0]}")

# Puedes eliminar o ajustar estos valores según el contexto
# Ejemplo: eliminar outliers
df = df[~((df['Income'] < (Q1 - 1.5 * IQR)) | (df['Income'] > (Q3 + 1.5 * IQR)))]

Selección de variables relevantes. No todas las variables aportan valor. Elimina las que no sean útiles (por ejemplo, ID).

Eliminamos identificadores y variables irrelevantes para el análisis.

(WRNNG: no ejecutar para conservar toda la info)

In [27]:
# Eliminar columnas que no aportan al análisis
df = df.drop(columns=['ID', 'Z_CostContact', 'Z_Revenue'])

Preprocesamiento
1. Tratamiento de valores faltantes
    - Si hay pocos valores faltantes en una columna, puedes eliminarlos (Codigo #1)
    - Si hay muchos, puedes imputar (rellenar) con la media, mediana o moda: (Codigo #2)


Se revisan los valores faltantes y se decide la estrategia según la cantidad y relevancia de los datos

In [ ]:
# Revisar valores faltantes por columna
df.isnull().sum()

In [ ]:
#Codigo #1: eliminar filas con valores faltantes en columnas críticas
df = df.dropna(subset=['NombreColumna'])

In [ ]:
#Codigo #2: imputar valores faltantes con la media, mediana o moda
df['Income'] = df['Income'].fillna(df['Income'].median())

2. Limpieza de variables categóricas



Las variables categóricas pueden tener errores de captura, valores raros o inconsistentes. Es importante agrupar o corregir estos valores para evitar problemas en el análisis y modelado.

Se revisaron y corrigieron valores raros o errores de captura en las variables categóricas. Las categorías poco frecuentes o incorrectas se agruparon bajo "Other" para evitar que afecten el análisis.

In [ ]:
# Ver los valores únicos de las variables categóricas principales
print("Valores únicos en Education:", df['Education'].unique())
print("Valores únicos en Marital_Status:", df['Marital_Status'].unique())

# Corregir errores de escritura y agrupar valores raros
df['Education'] = df['Education'].replace({'2n Cycle': '2nd Cycle'})
df['Marital_Status'] = df['Marital_Status'].replace({
    'Absurd': 'Other',
    'YOLO': 'Other',
    'Alone': 'Other'
})

# Verifica los cambios
print("Valores únicos corregidos en Education:", df['Education'].unique())
print("Valores únicos corregidos en Marital_Status:", df['Marital_Status'].unique())

3. Detección y tratamiento de outliers

Los outliers (valores extremos) pueden distorsionar los resultados de los modelos y análisis estadísticos. Es importante detectarlos y decidir si se eliminan, se corrigen o simplemente se reportan.

Se detectaron outliers en la variable Income utilizando el método del rango intercuartílico (IQR). Por el momento, no se eliminan para conservar la información completa, pero se reporta su cantidad y se visualizan con un boxplot.

In [ ]:
# Detección de outliers en la variable Income usando el método IQR
Q1 = df['Income'].quantile(0.25)
Q3 = df['Income'].quantile(0.75)
IQR = Q3 - Q1

# Definir límites para outliers
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

# Identificar outliers
outliers = df[(df['Income'] < lower_bound) | (df['Income'] > upper_bound)]
print(f"Número de outliers en Income: {outliers.shape[0]}")

# (Opcional) Visualizar los outliers
import matplotlib.pyplot as plt
plt.figure(figsize=(8,4))
plt.boxplot(df['Income'].dropna())
plt.title('Boxplot de Income')
plt.show()

4. Encoding de variables categóricas


Los modelos de machine learning requieren variables numéricas. El encoding convierte variables categóricas en variables numéricas (por ejemplo, usando one-hot encoding).

Se aplicó one-hot encoding a las variables categóricas Education y Marital_Status para convertirlas en variables numéricas, facilitando su uso en modelos de machine learning.

In [ ]:
# Aplicar one-hot encoding a las variables categóricas principales
df_encoded = pd.get_dummies(df, columns=['Education', 'Marital_Status'], drop_first=True)

# Verifica el resultado
df_encoded.head()

5. Escalamiento de variables numéricas

El escalamiento pone todas las variables numéricas en la misma escala, lo que es importante para muchos algoritmos de machine learning.

Se aplicó escalamiento estándar (media 0, desviación estándar 1) a las variables numéricas principales para mejorar el desempeño de los modelos y evitar que variables con mayor rango dominen el análisis.

In [ ]:
# Escalamiento de variables numéricasඞ

from sklearn.preprocessing import StandardScaler

# Asegúrate de que df_encoded esté definido
df_encoded = pd.get_dummies(df, columns=['Education', 'Marital_Status'], drop_first=True)

# Selecciona las columnas numéricas a escalar
cols_to_scale = [
    'Income', 'Recency', 'MntWines', 'MntFruits', 'MntMeatProducts', 'MntFishProducts',
    'MntSweetProducts', 'MntGoldProds', 'NumDealsPurchases', 'NumWebPurchases',
    'NumCatalogPurchases', 'NumStorePurchases', 'NumWebVisitsMonth'
]

# Aplica el escalador
scaler = StandardScaler()
df_encoded[cols_to_scale] = scaler.fit_transform(df_encoded[cols_to_scale])

# Verifica el resultado
df_encoded[cols_to_scale].describe()

In [53]:
#  (01_EDA_Preprocesamiento.ipynb)
df_encoded.to_csv('../data/df_encoded.csv', index=False)